## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall -y numpy pandas
!pip install numpy==1.26.4 pandas==2.2.2

# imgaug (compatible with older NumPy)
!pip install -q imgaug==0.4.0

# Downgrade NumPy to avoid issues with imgaug (NumPy ≥ 2.0 breaks imgaug)
!pip install -q numpy==1.26.4

# !pip install tensorflow==2.15.0

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate, UpSampling2D
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

from imgaug import augmenters as iaa ## for augmentation

## Defining paths

In [ ]:
model_name =  'deep_ResUNet_2500'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 15
EPOCHS = 70#100

TOTAL_DATA = 2500
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 123
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 5
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
# COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Augmentations

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Flipud(0.2),  # vertically flip 20% of the images
    iaa.Affine(
        scale={"x": (0.8, 1.1), "y": (0.8, 1.1)},  # zoom in or out (80-120%)
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # width & height shift (-20 to +20%)
        rotate=(-15, 15),  # rotation (-45 to 45 degrees)
        shear=(-16, 16)  # shear (-16 to 16 degrees)
    ),
    iaa.ElasticTransformation(alpha=(0, 2.0), sigma=0.25),  # apply elastic deformations
    iaa.GaussianBlur(sigma=(0, 2.0))  # apply Gaussian blur
])

In [ ]:
# Function to augment a batch of images and masks
def augment_batch(images, masks, seq, batch_size):
    augmented_images = []
    augmented_masks = []
    while len(augmented_images) < batch_size:
        aug_images, aug_masks = seq(images=images, segmentation_maps=np.expand_dims(masks, axis=-1))
        augmented_images.extend(aug_images)
        augmented_masks.extend(aug_masks)

        if len(augmented_images) >= batch_size:
            break

    # Ensure we only take as many as we need to reach the desired batch_size
    augmented_images = augmented_images[:batch_size]
    augmented_masks = augmented_masks[:batch_size]

    return np.array(augmented_images), np.squeeze(np.array(augmented_masks), axis=-1)

In [ ]:
num_augmented_images_needed = TOTAL_DATA  - len(x_train)  # Calculate how many augmented images we need
if num_augmented_images_needed > 0:
    augmented_x, augmented_y = augment_batch(x_train, y_train, seq, num_augmented_images_needed)
    # Concatenate the original and augmented datasets
    combined_x_train = np.concatenate((x_train, augmented_x), axis=0)
    combined_y_train = np.concatenate((y_train, augmented_y), axis=0)
else:
    combined_x_train = x_train
    combined_y_train = y_train

In [ ]:
x_train = combined_x_train
y_train = combined_y_train

## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 2500 original images into x_train
Loaded 2500 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )

    my_callbacks = [best_model_callback , reduce_LR]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### ResUnet ---Works with batch size20


In [ ]:
############################# CONVOLUTIONAL BLOCK #############################

def conv_block(feature_map):

    # Main Path
    conv_1 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(feature_map)
    bn = BatchNormalization()(conv_1)
    relu = Activation(activation='relu')(bn)
    conv_2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(relu)

    res_conn = Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same')(feature_map)
    res_conn = BatchNormalization()(res_conn)
    addition = add([res_conn, conv_2])

    return addition

############################### RESIDUAL BLOCK ################################

def res_block(feature_map, conv_filter, stride):

    bn_1 = BatchNormalization()(feature_map)
    relu_1 = Activation(activation='relu')(bn_1)
    conv_1 = Conv2D(conv_filter, kernel_size=(3,3), strides=stride[0], padding='same')(relu_1)
    bn_2 = BatchNormalization()(conv_1)
    relu_2 = Activation(activation='relu')(bn_2)
    conv_2 = Conv2D(conv_filter, kernel_size=(3,3), strides=stride[1], padding='same')(relu_2)


    res_conn = Conv2D(conv_filter, kernel_size=(1,1), strides=stride[0], padding='same')(feature_map)
    res_conn = BatchNormalization()(res_conn)
    addition = add([res_conn, conv_2])

    return addition

################################### ENCODER ###################################

def encoder(feature_map):

    # Initialize the to_decoder connection
    to_decoder = []

    # Block 1 - Convolution Block
    path = conv_block(feature_map)
    to_decoder.append(path)

    # Block 2 - Residual Block 1
    path = res_block(path, 128, [(2, 2), (1, 1)])
    to_decoder.append(path)

    # Block 3 - Residual Block 2
    path = res_block(path, 256, [(2, 2), (1, 1)])
    to_decoder.append(path)

    return to_decoder

################################### DECODER ###################################

def decoder(feature_map, from_encoder):

    # Block 1: Up-sample, Concatenation + Residual Block 1
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(feature_map)
    # main_path = Conv2DTranspose(filters=256, kernel_size=(2,2), strides=(2,2), padding='same')(feature_map)
    main_path = Concatenate()([main_path, from_encoder[2]])
    main_path = res_block(main_path, 256, [(1, 1), (1, 1)])

    # Block 2: Up-sample, Concatenation + Residual Block 2
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=128, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = Concatenate()([main_path, from_encoder[1]])
    main_path = res_block(main_path, 128, [(1, 1), (1, 1)])

    # Block 3: Up-sample, Concatenation + Residual Block 3
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=64, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = Concatenate()([main_path, from_encoder[0]])
    main_path = res_block(main_path, 64, [(1, 1), (1, 1)])

    return main_path

################################ RESIDUAL UNET ################################

def get_ResUNet():
    ## SOURCE: https://github.com/edwinpalegre/EE8204-ResUNet/blob/master/model_resunet.py

    # Input
    model_input = Input(shape=MODEL_INPUT_SIZE)

    # Encoder Path
    model_encoder = encoder(model_input)

    # Bottleneck
    model_bottleneck = res_block(model_encoder[2], 512, [(2, 2), (1, 1)])

    # Decoder Path
    model_decoder = decoder(model_bottleneck, model_encoder)

    # Output
    model_output = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1), activation='sigmoid', padding='same')(model_decoder)

    model= Model(model_input, model_output)

    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],
                  optimizer=optimizer)

    return model

## Train and Results

In [ ]:
# model= get_basic_unet() ## Model1
# model= get_unet_vgg16() ## Model2
# model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
model = get_ResUNet() ## Model5
# model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
# model = get_unet_InceptionResNetV2() ## Model8

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 256, 256,  │        256 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ add[0][0]         │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │     73,856 │ activation_1[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │      8,320 │ add[0][0]         │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat

 Total params: 8,233,025 (31.41 MB)

 Trainable params: 8,223,809 (31.37 MB)

 Non-trainable params: 9,216 (36.00 KB)

In [ ]:
# training the model and saving the best model as a check point
best_model_name, train_history, total_time = fit_and_save_best_model(model_name, model)

Epoch 1/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9372 - binary_io_u: 0.6249 - f1_score: 0.5435 - loss: 0.2038
Epoch 1: val_binary_io_u improved from -inf to 0.48117, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 281s 1s/step - accuracy: 0.9374 - binary_io_u: 0.6254 - f1_score: 0.5442 - loss: 0.2030 - val_accuracy: 0.9623 - val_binary_io_u: 0.4812 - val_f1_score: 1.1601e-04 - val_loss: 0.1379 - learning_rate: 0.0010
Epoch 2/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9803 - binary_io_u: 0.7781 - f1_score: 0.7218 - loss: 0.0596
Epoch 2: val_binary_io_u improved from 0.48117 to 0.49891, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9803 - binary_io_u: 0.7780 - f1_score: 0.7218 - loss: 0.0596 - val_accuracy: 0.9636 - val_binary_io_u: 0.4989 - val_f1_score: 0.0571 - val_loss: 0.1381 - learning_rate: 0.0010
Epoch 3/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9817 - binary_io_u: 0.7934 - f1_score: 0.7461 - loss: 0.0547
Epoch 3: val_binary_io_u improved from 0.49891 to 0.58374, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9817 - binary_io_u: 0.7934 - f1_score: 0.7461 - loss: 0.0547 - val_accuracy: 0.9696 - val_binary_io_u: 0.5837 - val_f1_score: 0.2928 - val_loss: 0.1021 - learning_rate: 0.0010
Epoch 4/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9823 - binary_io_u: 0.8000 - f1_score: 0.7562 - loss: 0.0526
Epoch 4: val_binary_io_u improved from 0.58374 to 0.73853, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9823 - binary_io_u: 0.7999 - f1_score: 0.7561 - loss: 0.0526 - val_accuracy: 0.9802 - val_binary_io_u: 0.7385 - val_f1_score: 0.6168 - val_loss: 0.0628 - learning_rate: 0.0010
Epoch 5/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9828 - binary_io_u: 0.8062 - f1_score: 0.7651 - loss: 0.0506
Epoch 5: val_binary_io_u improved from 0.73853 to 0.78731, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


...


Epoch 61: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
134/134 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.9928 - binary_io_u: 0.9140 - f1_score: 0.9053 - loss: 0.0185 - val_accuracy: 0.9901 - val_binary_io_u: 0.8747 - val_f1_score: 0.8339 - val_loss: 0.0301 - learning_rate: 1.2500e-04
Epoch 62/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9928 - binary_io_u: 0.9143 - f1_score: 0.9055 - loss: 0.0184
Epoch 62: val_binary_io_u did not improve from 0.87469
134/134 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9928 - binary_io_u: 0.9142 - f1_score: 0.9055 - loss: 0.0184 - val_accuracy: 0.9900 - val_binary_io_u: 0.8718 - val_f1_score: 0.8308 - val_loss: 0.0304 - learning_rate: 6.2500e-05
Epoch 63/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9928 - binary_io_u: 0.9146 - f1_score: 0.9061 - loss: 0.0181
Epoch 63: val_binary_io_u did not improve from 0.87469
134/134 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.9928 - binary_io_u: 0.9145 - f1

134/134 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9931 - binary_io_u: 0.9181 - f1_score: 0.9103 - loss: 0.0172 - val_accuracy: 0.9903 - val_binary_io_u: 0.8749 - val_f1_score: 0.8352 - val_loss: 0.0304 - learning_rate: 6.2500e-05
Epoch 66/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9932 - binary_io_u: 0.9193 - f1_score: 0.9117 - loss: 0.0169
Epoch 66: val_binary_io_u improved from 0.87489 to 0.87550, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9932 - binary_io_u: 0.9193 - f1_score: 0.9117 - loss: 0.0169 - val_accuracy: 0.9903 - val_binary_io_u: 0.8755 - val_f1_score: 0.8358 - val_loss: 0.0302 - learning_rate: 6.2500e-05
Epoch 67/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9933 - binary_io_u: 0.9204 - f1_score: 0.9130 - loss: 0.0166
Epoch 67: val_binary_io_u did not improve from 0.87550
134/134 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9933 - binary_io_u: 0.9204 - f1_score: 0.9129 - loss: 0.0166 - val_accuracy: 0.9903 - val_binary_io_u: 0.8753 - val_f1_score: 0.8358 - val_loss: 0.0308 - learning_rate: 6.2500e-05
Epoch 68/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9934 - binary_io_u: 0.9212 - f1_score: 0.9139 - loss: 0.0164
Epoch 68: val_binary_io_u improved from 0.87550 to 0.87610, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5



Epoch 68: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
134/134 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9934 - binary_io_u: 0.9211 - f1_score: 0.9139 - loss: 0.0164 - val_accuracy: 0.9904 - val_binary_io_u: 0.8761 - val_f1_score: 0.8365 - val_loss: 0.0300 - learning_rate: 6.2500e-05
Epoch 69/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9933 - binary_io_u: 0.9199 - f1_score: 0.9123 - loss: 0.0167
Epoch 69: val_binary_io_u improved from 0.87610 to 0.87885, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/deep_ResUNet_2500/saved_models/best_deep_ResUNet_2500.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.9933 - binary_io_u: 0.9198 - f1_score: 0.9123 - loss: 0.0167 - val_accuracy: 0.9905 - val_binary_io_u: 0.8789 - val_f1_score: 0.8413 - val_loss: 0.0289 - learning_rate: 3.1250e-05
Epoch 70/70
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9934 - binary_io_u: 0.9210 - f1_score: 0.9138 - loss: 0.0163
Epoch 70: val_binary_io_u did not improve from 0.87885
134/134 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.9934 - binary_io_u: 0.9210 - f1_score: 0.9137 - loss: 0.0163 - val_accuracy: 0.9906 - val_binary_io_u: 0.8788 - val_f1_score: 0.8413 - val_loss: 0.0290 - learning_rate: 3.1250e-05


## Loading the best model

In [ ]:
#Now, load the best model
best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
                                  custom_objects={'f1_score': f1_score,
                                                  'binary_io_u':iou_score_binary})

# best_model =model
# best_model = ks.models.load_model("/content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation/2500/deep_ResUNet_2500_batch10/saved_models/best_deep_ResUNet_2500_batch10.h5",
#                                   custom_objects={'f1_score': f1_score,
#                                                   'binary_io_u':iou_score_binary})



## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step


# AUC

In [ ]:
# Add this section *after* prediction is made with the best model
# i.e., after: predictions = best_model.predict(x_test, verbose=1)

from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9901, Std=0.0002, 95% CI=(0.9898, 0.9905)
